In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np 
import time
import glob
import sys
import cv2
import os

import torch

from cellpose import models, core, io, transforms
from cellpose.models import MODEL_NAMES

sys.path.insert(0, "../torchvf/")
from metrics import *

In [2]:
worm_dir       = "../data/bpcis/worm_test/"
bact_fluor_dir = "../data/bpcis/bact_fluor_test/"
bact_phase_dir = "../data/bpcis/bact_phase_test/"

In [3]:
worm_image_files = sorted(glob.glob(os.path.join(worm_dir, "*_img.tif")))
worm_masks_files = sorted(glob.glob(os.path.join(worm_dir, "*_masks.tif")))

bact_fluor_image_files = sorted(glob.glob(os.path.join(bact_fluor_dir, "*_img.tif")))
bact_fluor_masks_files = sorted(glob.glob(os.path.join(bact_fluor_dir, "*_masks.tif")))

bact_phase_image_files = sorted(glob.glob(os.path.join(bact_phase_dir, "*_img.tif")))
bact_phase_masks_files = sorted(glob.glob(os.path.join(bact_phase_dir, "*_masks.tif")))

In [4]:
worm_imgs  = [io.imread(f) for f in worm_image_files]
worm_masks = [cv2.imread(f, cv2.IMREAD_UNCHANGED) for f in worm_masks_files]

bact_fluor_imgs  = [io.imread(f) for f in bact_fluor_image_files]
bact_fluor_masks = [cv2.imread(f, cv2.IMREAD_UNCHANGED) for f in bact_fluor_masks_files]

bact_phase_imgs  = [io.imread(f) for f in bact_phase_image_files]
bact_phase_masks = [cv2.imread(f, cv2.IMREAD_UNCHANGED) for f in bact_phase_masks_files]

In [5]:
print(len(bact_phase_imgs), len(bact_phase_masks))
print(len(bact_fluor_imgs), len(bact_fluor_masks))
print(len(worm_imgs), len(worm_masks))

148 148
75 75
60 60


In [6]:
# During evaluation of the H1 and H2 models I didn't evaluate on certain images that 
# I believe have problems with ground truth masks. This makes sure I'm not evaluating 
# Cellpose or Omnipose on these too. These include images 3 and 5 from the bacterial 
# fluorescence subset and image 19 from the worm subset. 

def remove_indices(imgs, masks, indices):
    imgs  = [i for j, i in enumerate(imgs)  if j not in indices]
    masks = [i for j, i in enumerate(masks) if j not in indices]
    
    return imgs, masks

bact_fluor_imgs, bact_fluor_masks = remove_indices(bact_fluor_imgs, bact_fluor_masks, [3, 5])
worm_imgs, worm_masks = remove_indices(worm_imgs, worm_masks, [19])

In [7]:
print(len(bact_phase_imgs), len(bact_phase_masks))
print(len(bact_fluor_imgs), len(bact_fluor_masks))
print(len(worm_imgs), len(worm_masks))

148 148
73 73
59 59


In [8]:
print(MODEL_NAMES)

['cyto', 'nuclei', 'cyto2', 'bact_phase_cp', 'bact_fluor_cp', 'plant_cp', 'worm_cp', 'cyto2_omni', 'bact_phase_omni', 'bact_fluor_omni', 'plant_omni', 'worm_omni', 'worm_bact_omni', 'worm_high_res_omni']


In [9]:
chans = [0, 0]  # This means segment based on first channel, no second channel.

# Parameters:
mask_threshold = -1 
verbose = False     # Turn on if you want to see more output. 
use_GPU = True      # Defined above.
transparency = True # Transparency in flow output.
rescale = None      # Give this a number if you need to upscale or downscale your images.
omni = True         # We can turn off Omnipose mask reconstruction, not advised. 
flow_threshold = -1  # Default is .4, but only needed if there are spurious masks to clean up; slows down output.
resample = True      # Whether or not to run dynamics on rescaled grid or original grid. 


def eval_model(model, images, gt_masks):
    start = time.time()
    masks, flows, styles = model.eval(
        images, 
        channels = chans,
        rescale = rescale,
        mask_threshold = mask_threshold,
        transparency = transparency,
        flow_threshold = flow_threshold,
        omni = omni,
        resample = resample,
        verbose = verbose,
        cluster = True
    )

    out = map_iou(masks, gt_masks)

    f1s = []
    ious = []
    for pred_mask, true_mask in zip(masks, gt_masks):
        pred_mask = torch.Tensor(pred_mask[None][None] > 0)
        true_mask = torch.Tensor(true_mask[None][None] > 0)

        f1s.append(f1(pred_mask, true_mask).numpy())
        ious.append(iou(pred_mask, true_mask).numpy())

    print("F1 Score: ",  np.mean(f1s))
    print("IoU Score: ", np.mean(ious))
    
    print(f"AP_50: {out[0]:.3f} | AP_75: {out[1]:.3f} | AP_90: {out[2]:.3f} | AP: {np.mean(out):.3f}")

In [10]:
model = models.CellposeModel(
    gpu = use_GPU, 
    model_type = "worm_omni"
)

eval_model(model, worm_imgs, worm_masks)

2022-08-27 13:25:31,777 [INFO] >>worm_omni<< model set to be used
2022-08-27 13:25:32,997 [INFO] ** TORCH CUDA version installed and working. **
2022-08-27 13:25:32,998 [INFO] >>>> using GPU
2022-08-27 13:25:45,245 [INFO] 100%|##########| 59/59 [00:12<00:00,  4.87it/s]
F1 Score:  0.94117737
IoU Score:  0.8900828
AP_50: 0.813 | AP_75: 0.725 | AP_90: 0.375 | AP: 0.638


In [11]:
model = models.CellposeModel(
    gpu = use_GPU, 
    model_type = "bact_phase_omni"
)

eval_model(model, bact_phase_imgs, bact_phase_masks)

2022-08-27 13:25:45,736 [INFO] >>bact_phase_omni<< model set to be used
2022-08-27 13:25:45,738 [INFO] ** TORCH CUDA version installed and working. **
2022-08-27 13:25:45,738 [INFO] >>>> using GPU
2022-08-27 13:27:37,361 [INFO] 100%|##########| 148/148 [01:51<00:00,  1.33it/s]
F1 Score:  0.98023266
IoU Score:  0.9618153
AP_50: 0.942 | AP_75: 0.906 | AP_90: 0.780 | AP: 0.876


In [12]:
model = models.CellposeModel(
    gpu = use_GPU, 
    model_type = "bact_fluor_omni"
)

eval_model(model, bact_fluor_imgs, bact_fluor_masks)

2022-08-27 13:27:42,432 [INFO] >>bact_fluor_omni<< model set to be used
2022-08-27 13:27:42,434 [INFO] ** TORCH CUDA version installed and working. **
2022-08-27 13:27:42,435 [INFO] >>>> using GPU
2022-08-27 13:28:21,585 [INFO] 100%|##########| 73/73 [00:39<00:00,  1.87it/s]
F1 Score:  0.97905564
IoU Score:  0.959237
AP_50: 0.931 | AP_75: 0.893 | AP_90: 0.378 | AP: 0.734


In [13]:
model = models.CellposeModel(
    gpu = use_GPU, 
    model_type = "bact_phase_omni"
).net

2022-08-27 13:28:23,118 [INFO] >>bact_phase_omni<< model set to be used
2022-08-27 13:28:23,119 [INFO] ** TORCH CUDA version installed and working. **
2022-08-27 13:28:23,120 [INFO] >>>> using GPU


In [14]:
param_count = sum(p.numel() for p in model.parameters())

print(f"Model Parameter Count: {param_count}")

Model Parameter Count: 6600876
